## 1. Import Required Libraries

Import all necessary libraries for YOLO training, computer vision, and data visualization.

In [1]:
# Core Libraries
import os
import random
from pathlib import Path
import datetime
import time
import warnings
warnings.filterwarnings('ignore')

# YOLO and Deep Learning
from ultralytics import YOLO
import torch
import urllib
from pycocotools.coco import COCO
import shutil
from sklearn.model_selection import train_test_split

# Computer Vision
import cv2

# Data Science and Visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import yaml
from tqdm import tqdm

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("Training on CPU will be significantly slower than GPU")
    print("Consider using Google Colab or a GPU-enabled environment")

# Set matplotlib style
plt.style.use('default')
sns.set_palette("husl")

Using device: cuda
GPU: NVIDIA GeForce RTX 3060 Ti
CUDA Version: 12.8


## 2. Setup Project Directories and Configuration

Create necessary directories for custom dataset training and define training configuration.

In [ ]:
# Project Configuration
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
IMAGES_DIR = DATA_DIR / 'images'
LABELS_DIR = DATA_DIR / 'labels'
MODELS_DIR = PROJECT_ROOT / 'models'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'

# Create directories if they don't exist
for directory in [DATA_DIR, IMAGES_DIR, LABELS_DIR, MODELS_DIR, OUTPUTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Create train/val/test subdirectories
for split in ['train', 'val', 'test']:
    (IMAGES_DIR / split).mkdir(exist_ok=True)
    (LABELS_DIR / split).mkdir(exist_ok=True)

# Configuration dictionary - Training settings
config = {
    'model_name': 'yolov8n.pt',  # YOLOv8 Nano - mais estável
    'img_size': 640,
    'batch_size': 16,
    'epochs': 100,  # Aumentado para dar tempo ao modelo aprender
    'patience': 20,  # Early stopping após 20 épocas sem melhoria
    'confidence_threshold': 0.25,  # Threshold padrão (25% confiança mínima)
    'iou_threshold': 0.45,  # IOU padrão para NMS
    'dog_class_id': 0,  # Custom dataset - single class (dog)
    # Hiperparâmetros de treinamento otimizados
    'lr0': 0.01,  # Learning rate inicial
    'lrf': 0.01,  # Learning rate final (fração de lr0)
    'momentum': 0.937,  # SGD momentum
    'weight_decay': 0.0005,  # L2 regularization
    'warmup_epochs': 3,  # Épocas de warmup
    'warmup_momentum': 0.8,  # Momentum inicial durante warmup
    'warmup_bias_lr': 0.1,  # Learning rate de bias durante warmup
    'box': 7.5,  # Box loss gain
    'cls': 0.5,  # Class loss gain (baixo pois só temos 1 classe)
    'dfl': 1.5,  # DFL loss gain
    'hsv_h': 0.015,  # Augmentation: Hue
    'hsv_s': 0.7,  # Augmentation: Saturation
    'hsv_v': 0.4,  # Augmentation: Value
    'degrees': 0.0,  # Rotation (desabilitado para cachorros)
    'translate': 0.1,  # Translation
    'scale': 0.5,  # Scale
    'shear': 0.0,  # Shear (desabilitado)
    'perspective': 0.0,  # Perspective (desabilitado)
    'flipud': 0.0,  # Flip vertical (desabilitado para cachorros)
    'fliplr': 0.5,  # Flip horizontal (50% chance)
    'mosaic': 1.0,  # Mosaic augmentation
    'mixup': 0.0,  # Mixup augmentation (desabilitado para simplicidade)
    'copy_paste': 0.0  # Copy-paste augmentation (desabilitado)
}

print("Project Configuration:")
print(f"├── Data Directory: {DATA_DIR}")
print(f"├── Images Directory: {IMAGES_DIR}")
print(f"│   ├── train: {IMAGES_DIR / 'train'}")
print(f"│   ├── val: {IMAGES_DIR / 'val'}")
print(f"│   └── test: {IMAGES_DIR / 'test'}")
print(f"├── Labels Directory: {LABELS_DIR}")
print(f"│   ├── train: {LABELS_DIR / 'train'}")
print(f"│   ├── val: {LABELS_DIR / 'val'}")
print(f"│   └── test: {LABELS_DIR / 'test'}")
print(f"├── Models Directory: {MODELS_DIR}")
print(f"└── Outputs Directory: {OUTPUTS_DIR}")

print(f"\n{'='*60}")
print("TRAINING CONFIGURATION")
print(f"{'='*60}")
print(f"\nModel Settings:")
print(f"  Model:       {config['model_name']}")
print(f"  Image size:  {config['img_size']}px")
print(f"  Batch size:  {config['batch_size']}")
print(f"  Epochs:      {config['epochs']}")
print(f"  Patience:    {config['patience']} (early stopping)")

print(f"\nInference Settings:")
print(f"  Confidence:  {config['confidence_threshold']} (25% minimum)")
print(f"  IOU:         {config['iou_threshold']} (NMS threshold)")

print(f"\nLearning Settings:")
print(f"  Initial LR:  {config['lr0']}")
print(f"  Final LR:    {config['lrf']}")
print(f"  Momentum:    {config['momentum']}")
print(f"  Weight decay: {config['weight_decay']}")

print(f"\nData Augmentation:")
print(f"  HSV:         H={config['hsv_h']}, S={config['hsv_s']}, V={config['hsv_v']}")
print(f"  Flip LR:     {config['fliplr']*100:.0f}%")
print(f"  Translate:   {config['translate']*100:.0f}%")
print(f"  Scale:       {config['scale']*100:.0f}%")
print(f"  Mosaic:      {config['mosaic']*100:.0f}%")

print(f"\nTIP: These settings are optimized for single-class dog detection")
print(f"{'='*60}")

Project Configuration:
├── Data Directory: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data
├── Images Directory: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/images
│   ├── train: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/images/train
│   ├── val: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/images/val
│   └── test: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/images/test
├── Labels Directory: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/labels
│   ├── train: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/labels/train
│   ├── val: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/labels/val
│   └── test: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/labels/test
├── Models Directory: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/models
└── Outputs Directory: /home/joni/Docum

## 2.5. Download and Format COCO Dataset for Training (Optional)

Download COCO 2017 dataset, filter for dog images, and convert to YOLO format for training.

**What this section does:**
- Downloads COCO 2017 test dataset annotations
- Filters images containing dogs (COCO class_id: 18)
- Downloads filtered images
- Converts COCO bbox format to YOLO format
- Splits data into train (70%), val (20%), test (10%)
- Organizes into proper directory structure

**Note:** Skip this section if you already have a custom dataset prepared.

In [ ]:
# Configuration for COCO download
COCO_CONFIG = {
    'annotations_url': 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip',
    'images_base_url': 'http://images.cocodataset.org/train2017/',
    'dog_class_id': 18,  # Dog class in COCO dataset
    'max_samples': None,  # Maximum images to download (set None for all dog images)
    'train_split': 0.7, # Data to train the model
    'val_split': 0.2, # Data to validate the model during training
    'test_split': 0.1 # Data to test the model after training
}

def download_file(url, dest_path):
    """Download file with progress bar"""
    print(f"Downloading {url}...")
    try:
        with urllib.request.urlopen(url) as response:
            total_size = int(response.headers.get('content-length', 0))
            with open(dest_path, 'wb') as f, tqdm(total=total_size, unit='B', unit_scale=True) as pbar:
                while True:
                    chunk = response.read(8192)
                    if not chunk:
                        break
                    f.write(chunk)
                    pbar.update(len(chunk))
        print(f"Downloaded to {dest_path}")
        return True
    except Exception as e:
        print(f"Error downloading: {e}")
        return False

def coco_to_yolo_bbox(bbox, img_width, img_height):
    """
    Convert COCO bbox format to YOLO format
    COCO: [x_min, y_min, width, height] (absolute pixels)
    YOLO: [center_x, center_y, width, height] (normalized 0-1)
    """
    x_min, y_min, width, height = bbox
    
    # Calculate center coordinates
    center_x = (x_min + width / 2) / img_width
    center_y = (y_min + height / 2) / img_height
    
    # Normalize width and height
    norm_width = width / img_width
    norm_height = height / img_height
    
    return [center_x, center_y, norm_width, norm_height]

def download_and_format_coco_dogs():
    """Main function to download and format COCO dog dataset"""
    print("="*60)
    print("COCO Dog Dataset Download and Formatting")
    print("="*60)
    
    # Create temporary directory for annotations
    temp_dir = PROJECT_ROOT / 'temp_coco'
    temp_dir.mkdir(exist_ok=True)
    
    # Download annotations
    annotations_zip = temp_dir / 'annotations_trainval2017.zip'
    if not annotations_zip.exists():
        if not download_file(COCO_CONFIG['annotations_url'], annotations_zip):
            print("Failed to download annotations")
            return
        
        # Extract annotations
        print("Extracting annotations...")
        shutil.unpack_archive(annotations_zip, temp_dir)
    
    # Load COCO annotations
    annotations_file = temp_dir / 'annotations' / 'instances_train2017.json'
    print(f"\nLoading COCO annotations from {annotations_file}")
    coco = COCO(str(annotations_file))
    
    # Get all images with dogs
    dog_class_id = COCO_CONFIG['dog_class_id']
    dog_img_ids = coco.getImgIds(catIds=[dog_class_id])
    print(f"Found {len(dog_img_ids)} images containing dogs in COCO val2017")
    
    # Limit samples if specified
    if COCO_CONFIG['max_samples'] and len(dog_img_ids) > COCO_CONFIG['max_samples']:
        dog_img_ids = dog_img_ids[:COCO_CONFIG['max_samples']]
        print(f"Limiting to {COCO_CONFIG['max_samples']} images")
    
    # Get random non-dog images for test set (negative examples)
    all_img_ids = coco.getImgIds()
    non_dog_img_ids = [img_id for img_id in all_img_ids if img_id not in dog_img_ids]
    
    # Sample random non-dog images (about 50% of test set will be non-dog images)
    num_test_dogs = int(len(dog_img_ids) * COCO_CONFIG['test_split'])
    num_non_dog_test = num_test_dogs  # Equal number = 50% each
    random.seed(42)
    non_dog_test_ids = random.sample(non_dog_img_ids, min(num_non_dog_test, len(non_dog_img_ids)))
    print(f"Selected {len(non_dog_test_ids)} random non-dog images for test set (50% of test data)")
    
    # Split into train/val/test
    train_ratio = COCO_CONFIG['train_split']
    val_ratio = COCO_CONFIG['val_split']
    test_ratio = COCO_CONFIG['test_split']
    
    # First split: separate test set (only from dog images)
    train_val_ids, test_dog_ids = train_test_split(
        dog_img_ids, 
        test_size=test_ratio, 
        random_state=42
    )
    
    # Add non-dog images to test set
    test_ids = test_dog_ids + non_dog_test_ids
    
    # Second split: separate train and val
    val_size = val_ratio / (train_ratio + val_ratio)
    train_ids, val_ids = train_test_split(
        train_val_ids, 
        test_size=val_size, 
        random_state=42
    )
    
    splits = {
        'train': train_ids,
        'val': val_ids,
        'test': test_ids
    }
    
    print(f"\nDataset splits:")
    print(f"  Train: {len(train_ids)} images ({train_ratio*100:.0f}%) - all with dogs")
    print(f"  Val:   {len(val_ids)} images ({val_ratio*100:.0f}%) - all with dogs")
    print(f"  Test:  {len(test_ids)} images ({len(test_dog_ids)} dogs + {len(non_dog_test_ids)} non-dogs)")
    
    # Process each split
    for split_name, img_ids in splits.items():
        print(f"\nProcessing {split_name} set...")
        images_split_dir = IMAGES_DIR / split_name
        labels_split_dir = LABELS_DIR / split_name
        
        # Clear existing files in split
        for f in images_split_dir.glob('*'):
            f.unlink()
        for f in labels_split_dir.glob('*.txt'):
            f.unlink()
        
        for img_id in tqdm(img_ids, desc=f"Downloading {split_name}"):
            # Get image info
            img_info = coco.loadImgs(img_id)[0]
            img_filename = img_info['file_name']
            img_width = img_info['width']
            img_height = img_info['height']
            
            # Download image
            img_url = COCO_CONFIG['images_base_url'] + img_filename
            img_path = images_split_dir / img_filename
            
            try:
                urllib.request.urlretrieve(img_url, img_path)
            except Exception as e:
                print(f"Failed to download {img_filename}: {e}")
                continue
            
            # Get annotations for this image (only dogs)
            ann_ids = coco.getAnnIds(imgIds=img_id, catIds=[dog_class_id])
            annotations = coco.loadAnns(ann_ids)
            
            # Convert to YOLO format and save
            label_path = labels_split_dir / img_filename.replace('.jpg', '.txt')
            
            # For non-dog images (negative examples), create empty label file
            if len(annotations) == 0:
                # Create empty label file for images with no dogs
                label_path.touch()
            else:
                # Write dog annotations
                with open(label_path, 'w') as f:
                    for ann in annotations:
                        bbox = ann['bbox']
                        yolo_bbox = coco_to_yolo_bbox(bbox, img_width, img_height)
                        # YOLO format: class_id center_x center_y width height
                        f.write(f"0 {yolo_bbox[0]:.6f} {yolo_bbox[1]:.6f} {yolo_bbox[2]:.6f} {yolo_bbox[3]:.6f}\n")
    
    # Clean up temporary files
    print("\nCleaning up temporary files...")
    shutil.rmtree(temp_dir)
    
    print("\n" + "="*60)
    print("COCO Dataset Download Complete!")
    print("="*60)
    print(f"Images saved to: {IMAGES_DIR}")
    print(f"Labels saved to: {LABELS_DIR}")
    print("\nDataset is ready for training!")
    
    return True

# Option to download COCO dataset
print("COCO Dog Dataset Preparation")
print("-" * 60)
print("This will download COCO 2017 validation images containing dogs")
print(f"and format them for YOLO training.\n")
print(f"Configuration:")
print(f"  - Max samples: {COCO_CONFIG['max_samples']}")
print(f"  - Train/Val/Test split: {COCO_CONFIG['train_split']}/{COCO_CONFIG['val_split']}/{COCO_CONFIG['test_split']}")
print(f"  - Target directory: {DATA_DIR}")
print("\nTo download and format COCO dataset, run:")
print("  download_and_format_coco_dogs()")
print("\nOr skip this cell if you have your own dataset ready.")

# Uncomment the line below to automatically download COCO dataset
download_and_format_coco_dogs()

## 3. Define Helper Functions

Create utility functions for image processing, visualization, training metrics, and dog detection.

In [3]:
def visualize_detections(image_path, results, save_path=None):
    """Visualize detection results with bounding boxes"""
    # Read image
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Plot results
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(img)
    
    # Draw bounding boxes
    if results and len(results[0].boxes) > 0:
        for box in results[0].boxes:
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            confidence = box.conf[0].cpu().numpy()
            class_id = int(box.cls[0].cpu().numpy())
            
            # Draw rectangle
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                                fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)
            
            # Add label
            label = f"Dog {confidence:.2f}"
            ax.text(x1, y1-10, label, color='red', fontsize=12,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.axis('off')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        print(f"Saved visualization to {save_path}")
    
    plt.show()
    return fig

def process_video_detections(video_path, model, output_path, conf_threshold=0.25):
    """Process video file and detect dogs frame by frame"""
    cap = cv2.VideoCapture(str(video_path))
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
    
    print(f"Processing video: {total_frames} frames at {fps} FPS")
    
    frame_count = 0
    with tqdm(total=total_frames) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run detection
            results = model(frame, conf=conf_threshold, verbose=False)
            
            # Draw results on frame
            annotated_frame = results[0].plot()
            
            # Write frame
            out.write(annotated_frame)
            
            frame_count += 1
            pbar.update(1)
    
    cap.release()
    out.release()
    
    print(f"Processed {frame_count} frames. Output saved to {output_path}")
    return output_path

def display_metrics(results_df):
    """Display training metrics and create visualizations"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot losses
    if 'train/box_loss' in results_df.columns:
        axes[0, 0].plot(results_df['epoch'], results_df['train/box_loss'], label='Box Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_title('Training Box Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
    
    # Plot mAP
    if 'metrics/mAP50(B)' in results_df.columns:
        axes[0, 1].plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP@0.5')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('mAP')
        axes[0, 1].set_title('Mean Average Precision')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
    
    # Plot precision and recall
    if 'metrics/precision(B)' in results_df.columns:
        axes[1, 0].plot(results_df['epoch'], results_df['metrics/precision(B)'], label='Precision')
        axes[1, 0].plot(results_df['epoch'], results_df['metrics/recall(B)'], label='Recall')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Score')
        axes[1, 0].set_title('Precision and Recall')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    # Summary statistics
    axes[1, 1].axis('off')
    if not results_df.empty:
        summary_text = f"Training Summary\n\n"
        summary_text += f"Total Epochs: {len(results_df)}\n"
        if 'metrics/mAP50(B)' in results_df.columns:
            summary_text += f"Best mAP@0.5: {results_df['metrics/mAP50(B)'].max():.4f}\n"
        if 'metrics/precision(B)' in results_df.columns:
            summary_text += f"Best Precision: {results_df['metrics/precision(B)'].max():.4f}\n"
        if 'metrics/recall(B)' in results_df.columns:
            summary_text += f"Best Recall: {results_df['metrics/recall(B)'].max():.4f}\n"
        
        axes[1, 1].text(0.1, 0.5, summary_text, fontsize=14, family='monospace',
                       verticalalignment='center')
    
    plt.tight_layout()
    plt.show()
    
    return fig

print("Helper functions defined successfully!")

Helper functions defined successfully!


## 4. Load Base YOLO Model

Initialize YOLO model with pre-trained COCO weights as a starting point for fine-tuning.

In [37]:
# Load pre-trained YOLO model (will be fine-tuned)
print(f"Loading {config['model_name']} model as base...")
model = YOLO(config['model_name'])

# Display model information
print(f"\nBase model loaded successfully!")
print(f"Model type: {type(model)}")
print(f"Device: {model.device}")
print("\nThis model will be fine-tuned on your custom dog dataset.")
print("The pre-trained weights provide a good starting point for training.")

Loading yolo11n.pt model as base...

Base model loaded successfully!
Model type: <class 'ultralytics.models.yolo.model.YOLO'>
Device: cpu

This model will be fine-tuned on your custom dog dataset.
The pre-trained weights provide a good starting point for training.


## 5. Prepare Custom Dataset for Training

Set up the dataset configuration file and verify dataset structure.

**Required**: Your dataset must follow YOLO format:
- Images in `data/images/train/`, `data/images/val/`, `data/images/test/`
- Labels in `data/labels/train/`, `data/labels/val/`, `data/labels/test/`
- Each image has a corresponding `.txt` label file
- Label format: `class_id center_x center_y width height` (all normalized 0-1)

In [38]:
# Create dataset configuration file for training
dataset_config = {
    'path': str(DATA_DIR.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': {
        0: 'dog'
    },
    'nc': 1  # number of classes
}

# Save dataset configuration
dataset_yaml_path = DATA_DIR / 'dataset.yaml'
with open(dataset_yaml_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print(f"Dataset configuration saved to: {dataset_yaml_path}")
print("\nDataset Configuration:")
print(yaml.dump(dataset_config, default_flow_style=False))

# Verify dataset structure
print("\nVerifying dataset structure...")
train_images = list((IMAGES_DIR / 'train').glob('*.*'))
train_labels = list((LABELS_DIR / 'train').glob('*.txt'))
val_images = list((IMAGES_DIR / 'val').glob('*.*'))
val_labels = list((LABELS_DIR / 'val').glob('*.txt'))
test_images = list((IMAGES_DIR / 'test').glob('*.*'))
test_labels = list((LABELS_DIR / 'test').glob('*.txt'))

print(f"\nDataset Summary:")
print(f"├── Training set:")
print(f"│   ├── Images: {len(train_images)}")
print(f"│   └── Labels: {len(train_labels)}")
print(f"├── Validation set:")
print(f"│   ├── Images: {len(val_images)}")
print(f"│   └── Labels: {len(val_labels)}")
print(f"└── Test set:")
print(f"    ├── Images: {len(test_images)}")
print(f"    └── Labels: {len(test_labels)}")

# Check if dataset is ready
if len(train_images) > 0 and len(val_images) > 0:
    print(f"\nDataset is ready for training!")
    if len(train_labels) != len(train_images):
        print(f"Warning: Number of training labels ({len(train_labels)}) doesn't match images ({len(train_images)})")
    if len(val_labels) != len(val_images):
        print(f"Warning: Number of validation labels ({len(val_labels)}) doesn't match images ({len(val_images)})")
else:
    print(f"\nDataset not ready!")
    print(f"   Please add labeled images to:")
    print(f"   - Training: {IMAGES_DIR / 'train'} and {LABELS_DIR / 'train'}")
    print(f"   - Validation: {IMAGES_DIR / 'val'} and {LABELS_DIR / 'val'}")

Dataset configuration saved to: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/dataset.yaml

Dataset Configuration:
names:
  0: dog
nc: 1
path: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data
test: images/test
train: images/train
val: images/val


Verifying dataset structure...

Dataset Summary:
├── Training set:
│   ├── Images: 349
│   └── Labels: 349
├── Validation set:
│   ├── Images: 101
│   └── Labels: 101
└── Test set:
    ├── Images: 100
    └── Labels: 100

Dataset is ready for training!


## 5.5. Dataset Quality Check

Analyze dataset quality before training to identify potential issues.

In [ ]:
def analyze_dataset_quality(split_name):
    """Analyze label quality for a dataset split"""
    labels_dir = LABELS_DIR / split_name
    images_dir = IMAGES_DIR / split_name
    
    label_files = list(labels_dir.glob('*.txt'))
    image_files = list(images_dir.glob('*.*'))
    
    stats = {
        'total_labels': len(label_files),
        'total_images': len(image_files),
        'empty_labels': 0,
        'labels_with_objects': 0,
        'total_objects': 0,
        'objects_per_image': [],
        'bbox_issues': 0
    }
    
    for label_file in label_files:
        with open(label_file, 'r') as f:
            lines = f.readlines()
        
        if len(lines) == 0:
            stats['empty_labels'] += 1
        else:
            stats['labels_with_objects'] += 1
            stats['total_objects'] += len(lines)
            stats['objects_per_image'].append(len(lines))
            
            # Check bbox format
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    stats['bbox_issues'] += 1
                    continue
                try:
                    class_id, cx, cy, w, h = map(float, parts)
                    # Check if normalized (0-1)
                    if not (0 <= cx <= 1 and 0 <= cy <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
                        stats['bbox_issues'] += 1
                except:
                    stats['bbox_issues'] += 1
    
    return stats

print("="*60)
print("DATASET QUALITY ANALYSIS")
print("="*60)

for split in ['train', 'val', 'test']:
    stats = analyze_dataset_quality(split)
    
    print(f"\n{split.upper()} Split:")
    print(f"  Total images:  {stats['total_images']}")
    print(f"  Total labels:  {stats['total_labels']}")
    print(f"  Images with objects: {stats['labels_with_objects']} ({stats['labels_with_objects']/stats['total_labels']*100:.1f}%)")
    print(f"  Empty labels (negatives): {stats['empty_labels']} ({stats['empty_labels']/stats['total_labels']*100:.1f}%)")
    print(f"  Total objects detected: {stats['total_objects']}")
    
    if stats['objects_per_image']:
        avg_objects = sum(stats['objects_per_image']) / len(stats['objects_per_image'])
        max_objects = max(stats['objects_per_image'])
        print(f"  Avg objects/image: {avg_objects:.2f}")
        print(f"  Max objects/image: {max_objects}")
    
    if stats['bbox_issues'] > 0:
        print(f"  Bbox format issues: {stats['bbox_issues']}")

# Recommendations
print(f"\n{'='*60}")
print("RECOMMENDATIONS")
print("="*60)

train_stats = analyze_dataset_quality('train')
val_stats = analyze_dataset_quality('val')
test_stats = analyze_dataset_quality('test')

# Check for negative samples in training
if train_stats['empty_labels'] > 0:
    neg_ratio = train_stats['empty_labels'] / train_stats['total_labels'] * 100
    print(f"\nTraining set contains {neg_ratio:.1f}% negative samples (no objects)")
    if neg_ratio > 10:
        print(f"   Recommendation: Remove or reduce negative samples in training")
        print(f"   Negative samples are better suited for validation/test sets")

# Check for negative samples in test
if test_stats['empty_labels'] > 0:
    neg_ratio = test_stats['empty_labels'] / test_stats['total_labels'] * 100
    print(f"\nTest set contains {neg_ratio:.1f}% negative samples")
    if neg_ratio > 60:
        print(f"   Too many negatives! This will skew validation metrics")
        print(f"   Recommendation: Aim for 10-20% negative samples in test set")
    elif neg_ratio < 5:
        print(f"   Too few negatives! Add more negative samples for robust validation")

# Check dataset size
if train_stats['total_images'] < 100:
    print(f"\nSmall training set ({train_stats['total_images']} images)")
    print(f"   Recommendation: Collect more training data (aim for 500+ images)")
    print(f"   Consider using data augmentation to increase effective dataset size")

if train_stats['total_objects'] < 200:
    print(f"\nFew training objects ({train_stats['total_objects']} total)")
    print(f"   Recommendation: Increase training data for better model performance")

# Check bbox issues
total_bbox_issues = train_stats['bbox_issues'] + val_stats['bbox_issues'] + test_stats['bbox_issues']
if total_bbox_issues > 0:
    print(f"\nBbox format issues detected: {total_bbox_issues} total")
    print(f"   Action required: Fix label format before training!")
else:
    print(f"\nAll bounding boxes are properly formatted")

print("="*60)

DATASET QUALITY ANALYSIS

📂 TRAIN Split:
  Total images:  349
  Total labels:  349
  Images with objects: 349 (100.0%)
  Empty labels (negatives): 0 (0.0%)
  Total objects detected: 473
  Avg objects/image: 1.36
  Max objects/image: 20

📂 VAL Split:
  Total images:  101
  Total labels:  101
  Images with objects: 101 (100.0%)
  Empty labels (negatives): 0 (0.0%)
  Total objects detected: 117
  Avg objects/image: 1.16
  Max objects/image: 4

📂 TEST Split:
  Total images:  100
  Total labels:  100
  Images with objects: 51 (51.0%)
  Empty labels (negatives): 49 (49.0%)
  Total objects detected: 59
  Avg objects/image: 1.16
  Max objects/image: 4

📋 RECOMMENDATIONS

✅ Test set contains 49.0% negative samples

✅ All bounding boxes are properly formatted


## 6. Train Custom Model

Fine-tune YOLOv8 on your custom dog dataset with optimized hyperparameters.

In [ ]:
# Check if training data exists
train_images = list((IMAGES_DIR / 'train').glob('*.*'))
val_images = list((IMAGES_DIR / 'val').glob('*.*'))

print(f"Training images found: {len(train_images)}")
print(f"Validation images found: {len(val_images)}")

if len(train_images) > 0 and len(val_images) > 0:
    print("\n" + "="*60)
    print("STARTING MODEL TRAINING")
    print("="*60)
    print(f"\nDataset:")
    print(f"  Training:   {len(train_images)} images")
    print(f"  Validation: {len(val_images)} images")
    
    print(f"\nTraining Configuration:")
    print(f"  Device:     {device}")
    print(f"  Model:      {config['model_name']}")
    print(f"  Epochs:     {config['epochs']}")
    print(f"  Patience:   {config['patience']} (early stopping)")
    print(f"  Batch size: {config['batch_size']}")
    print(f"  Image size: {config['img_size']}px")
    print(f"  Initial LR: {config['lr0']}")
    
    print(f"\nEstimated time:")
    if device == 'cuda':
        print(f"  ~2-4 hours on GPU for {len(train_images)} images")
    else:
        print(f"  ~8-12 hours on CPU for {len(train_images)} images")
        print(f"  Consider using GPU for faster training!")
    
    print("\n" + "="*60)
    print("Training in progress...")
    print("="*60 + "\n")
    
    # Start training with optimized hyperparameters
    results = model.train(
        # Dataset
        data=str(dataset_yaml_path),
        
        # Training duration
        epochs=config['epochs'],
        patience=config['patience'],
        
        # Model & hardware
        imgsz=config['img_size'],
        batch=config['batch_size'],
        device=device,
        
        # Output
        name='yolo_dog_custom',
        project=str(MODELS_DIR),
        save=True,
        save_period=-1,  # Save checkpoint every N epochs (-1 = only save last)
        
        # Visualization
        plots=True,
        verbose=True,
        
        # Optimization
        optimizer='auto',  # AdamW ou SGD automático
        lr0=config['lr0'],
        lrf=config['lrf'],
        momentum=config['momentum'],
        weight_decay=config['weight_decay'],
        warmup_epochs=config['warmup_epochs'],
        warmup_momentum=config['warmup_momentum'],
        warmup_bias_lr=config['warmup_bias_lr'],
        
        # Loss gains
        box=config['box'],
        cls=config['cls'],
        dfl=config['dfl'],
        
        # Data augmentation
        hsv_h=config['hsv_h'],
        hsv_s=config['hsv_s'],
        hsv_v=config['hsv_v'],
        degrees=config['degrees'],
        translate=config['translate'],
        scale=config['scale'],
        shear=config['shear'],
        perspective=config['perspective'],
        flipud=config['flipud'],
        fliplr=config['fliplr'],
        mosaic=config['mosaic'],
        mixup=config['mixup'],
        copy_paste=config['copy_paste'],
        
        # Validation
        val=True,
        fraction=1.0,  # Usar 100% dos dados
        
        # Performance
        cache=False,  # Não cachear imagens em memória (economiza RAM)
        workers=8,  # Número de workers para DataLoader
        
        # Other
        seed=42,  # Reprodutibilidade
        deterministic=True,  # Resultados determinísticos
        single_cls=True,  # Otimização para single-class
        rect=False,  # Rectangular training (desabilitado)
        cos_lr=False,  # Cosine LR scheduler (desabilitado)
        close_mosaic=10,  # Desabilitar mosaic nos últimos N epochs
        resume=False,  # Não retomar treinamento anterior
        amp=True,  # Automatic Mixed Precision (mais rápido em GPU)
        overlap_mask=True,
        mask_ratio=4,
        dropout=0.0,
        nbs=64  # Nominal batch size
    )
    
    print("\n" + "="*60)
    print("TRAINING COMPLETED!")
    print("="*60)
    print(f"\nModel saved to: {MODELS_DIR / 'yolo_dog_custom'}")
    print(f"  ├── Best weights:  weights/best.pt")
    print(f"  ├── Last weights:  weights/last.pt")
    print(f"  ├── Metrics:       results.csv")
    print(f"  └── Plots:         *.png")
    
    # Load best model for inference
    best_model_path = MODELS_DIR / 'yolo_dog_custom' / 'weights' / 'best.pt'
    if best_model_path.exists():
        model = YOLO(str(best_model_path))
        print(f"\nLoaded best model for inference: {best_model_path.name}")
        
        # Show final metrics
        results_csv = MODELS_DIR / 'yolo_dog_custom' / 'results.csv'
        if results_csv.exists():
            df = pd.read_csv(results_csv)
            print(f"\nFinal Training Metrics:")
            print(f"  Epochs trained:  {len(df)}")
            print(f"  Best mAP@0.5:    {df['metrics/mAP50(B)'].max():.4f}")
            print(f"  Best Precision:  {df['metrics/precision(B)'].max():.4f}")
            print(f"  Best Recall:     {df['metrics/recall(B)'].max():.4f}")
    
    print("\n" + "="*60)
    
else:
    print("\n" + "="*60)
    print("NO TRAINING DATA FOUND")
    print("="*60)
    print("\nPlease add labeled images to train the model:")
    print(f"\nRequired directories:")
    print(f"  Training images:  {IMAGES_DIR / 'train'}")
    print(f"  Training labels:  {LABELS_DIR / 'train'}")
    print(f"  Validation images: {IMAGES_DIR / 'val'}")
    print(f"  Validation labels: {LABELS_DIR / 'val'}")
    
    print(f"\nLabel Format (YOLO):")
    print(f"  Each .txt file should contain:")
    print(f"  class_id center_x center_y width height")
    print(f"\n  Example:")
    print(f"  0 0.5 0.5 0.3 0.4")
    print(f"\n  (All values normalized 0-1 relative to image dimensions)")
    print("="*60)

Training images found: 349
Validation images found: 101

STARTING MODEL TRAINING

📊 Dataset:
  Training:   349 images
  Validation: 101 images

⚙️  Training Configuration:
  Device:     cuda
  Model:      yolov8n.pt
  Epochs:     100
  Patience:   20 (early stopping)
  Batch size: 16
  Image size: 640px
  Initial LR: 0.01

⏱️  Estimated time:
  ~2-4 hours on GPU for 349 images

Training in progress...


✅ TRAINING COMPLETED!

📁 Model saved to: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/models/yolo_dog_custom
  ├── Best weights:  weights/best.pt
  ├── Last weights:  weights/last.pt
  ├── Metrics:       results.csv
  └── Plots:         *.png

✅ Loaded best model for inference: best.pt

📊 Final Training Metrics:
  Epochs trained:  100
  Best mAP@0.5:    0.7037
  Best Precision:  0.8625
  Best Recall:     0.7238



---

# Model execution

## 7. Video Processing with Trained Model

Process video files using your trained custom model to detect dogs in real-time video streams.

In [ ]:
# Check for video files
video_extensions = ['*.mp4', '*.avi', '*.mov', '*.mkv']
video_files = []
for ext in video_extensions:
    video_files.extend(IMAGES_DIR.glob(ext))

print(f"Video files found: {len(video_files)}")
for video in video_files:
    print(f"  - {video.name}")

if video_files:
    for video_path in video_files:
        print(f"\nProcessing video: {video_path.name}")
        output_path = OUTPUTS_DIR / f"detected_custom_{video_path.name}"
        
        process_video_detections(
            video_path=video_path,
            model=model,
            output_path=output_path,
            conf_threshold=config['confidence_threshold']
        )
        
        print(f"Output video saved to: {output_path}")
else:
    print("\nNo video files found.")
    print(f"Add video files to: {IMAGES_DIR}")
    print("\nSupported formats: MP4, AVI, MOV, MKV")

## 8. Batch Prediction on Custom Images

Run inference on multiple images and save results.

In [30]:
# Batch prediction on all images
def batch_predict(image_dir, output_dir, model, conf_threshold=0.25):
    """Run prediction on all images in a directory"""
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    all_images = []
    for ext in image_extensions:
        all_images.extend(Path(image_dir).glob(ext))
    
    if not all_images:
        print(f"No images found in {image_dir}")
        return None
    
    print(f"Processing {len(all_images)} images...")
    results_data = []
    
    for img_path in tqdm(all_images):
        # Run inference
        results = model(
            str(img_path),
            conf=conf_threshold,
            verbose=False
        )
        
        # Count detections
        num_dogs = len(results[0].boxes)
        
        # Save results
        results_data.append({
            'image': img_path.name,
            'dogs_detected': num_dogs,
            'confidence_avg': results[0].boxes.conf.mean().item() if num_dogs > 0 else 0
        })
        
        # Save annotated image
        if num_dogs > 0:
            output_path = Path(output_dir) / f"detected_custom_{img_path.name}"
            annotated = results[0].plot()
            cv2.imwrite(str(output_path), annotated)
    
    # Create results dataframe
    results_df = pd.DataFrame(results_data)
    return results_df

# Run batch prediction on validation set
val_images = list((PROJECT_ROOT / 'test').glob('*.*'))

if val_images:
    print("Running batch prediction on validation images...\n")
    results_df = batch_predict(PROJECT_ROOT / 'test', OUTPUTS_DIR, model, config['confidence_threshold'])
    
    if results_df is not None:
        print("\nBatch Prediction Results:")
        print(results_df.to_string(index=False))
        
        print(f"\nSummary:")
        print(f"  Total images processed: {len(results_df)}")
        print(f"  Images with dogs detected: {(results_df['dogs_detected'] > 0).sum()}")
        print(f"  Total dogs detected: {results_df['dogs_detected'].sum()}")
        print(f"  Average confidence: {results_df['confidence_avg'].mean():.3f}")
        
        # Save results to CSV
        results_csv = OUTPUTS_DIR / 'detection_results_custom.csv'
        results_df.to_csv(results_csv, index=False)
        print(f"\nResults saved to: {results_csv}")
else:
    print("No validation images available for batch prediction.")
    print(f"Add images to: {IMAGES_DIR / 'val'}")

Running batch prediction on validation images...

Processing 2 images...


100%|██████████| 2/2 [00:00<00:00, 31.96it/s]


Batch Prediction Results:
           image  dogs_detected  confidence_avg
000000106525.jpg              0               0
000000106850.jpg              0               0

Summary:
  Total images processed: 2
  Images with dogs detected: 0
  Total dogs detected: 0
  Average confidence: 0.000

Results saved to: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/outputs/detection_results_custom.csv


## 9. Model Evaluation and Metrics

Evaluate trained model performance on the test dataset with ground truth labels.

In [46]:
# Check for test dataset
test_images = list((IMAGES_DIR / 'test').glob('*.*'))
test_labels = list((LABELS_DIR / 'test').glob('*.txt'))

print(f"Test images: {len(test_images)}")
print(f"Test labels: {len(test_labels)}")

if len(test_images) > 0 and len(test_labels) > 0:
    print("\nTest dataset found!")
    print("Running model evaluation on test set...\n")
    
    # Run validation on test set
    metrics = model.val(
        data=str(dataset_yaml_path),
        split='test',
        imgsz=config['img_size'],
        batch=config['batch_size'],
        conf=config['confidence_threshold'],
        iou=config['iou_threshold'],
        device=device
    )
    
    # Display metrics
    print("\n" + "="*60)
    print("Evaluation Metrics")
    print("="*60)
    print(f"  mAP@0.5:      {metrics.box.map50:.4f}")
    print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
    print(f"  Precision:    {metrics.box.mp:.4f}")
    print(f"  Recall:       {metrics.box.mr:.4f}")
    print("="*60)
    
    # Plot confusion matrix if available
    if hasattr(metrics, 'confusion_matrix') and metrics.confusion_matrix is not None:
        plt.figure(figsize=(8, 6))
        sns.heatmap(metrics.confusion_matrix.matrix, annot=True, fmt='g', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        
        cm_path = OUTPUTS_DIR / 'confusion_matrix_custom.png'
        plt.savefig(cm_path)
        print(f"\nConfusion matrix saved to: {cm_path}")
        plt.show()
else:
    print("\nNo test dataset found.")
    print("To evaluate the model, add test images and labels:")
    print(f"  - Images: {IMAGES_DIR / 'test'}")
    print(f"  - Labels: {LABELS_DIR / 'test'}")
    print("\nTest set evaluation requires labeled ground truth data.")

Test images: 100
Test labels: 100

Test dataset found!
Running model evaluation on test set...

Ultralytics 8.3.232 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7831MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4621.8±1218.0 MB/s, size: 90.6 KB)
val: Scanning /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/labels/test.cache... 100 images, 49 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 380.6Kit/s 0.0s
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4621.8±1218.0 MB/s, size: 90.6 KB)
val: Scanning /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/data/labels/test.cache... 100 images, 49 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 380.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━

<Figure size 800x600 with 2 Axes>

## 10. Model Export and Deployment

Export the trained model for deployment and run inference demonstrations.

**Export Formats Supported:**
- **ONNX**: Cross-platform inference

This section demonstrates model export and compares inference performance.

In [ ]:
# Check if trained model exists
best_model_path = MODELS_DIR / 'yolo_dog_custom' / 'weights' / 'best.pt'
last_model_path = MODELS_DIR / 'yolo_dog_custom' / 'weights' / 'last.pt'

if best_model_path.exists():
    print("="*60)
    print("Model Export and Deployment")
    print("="*60)
    
    # Load the best trained model
    print(f"\nLoading best trained model from: {best_model_path}")
    export_model = YOLO(str(best_model_path))
    print(f"Model loaded successfully!")
    
    # Create exports directory
    exports_dir = OUTPUTS_DIR / 'exports'
    exports_dir.mkdir(exist_ok=True)
    
    print(f"\nExport directory: {exports_dir}")
    print("-" * 60)

    print("Exporting model to ONNX format...")
    print("-" * 60)
    
    try:
        # Export to ONNX
        onnx_path = export_model.export(
            format='onnx',
            imgsz=config['img_size'],
            simplify=True,  # Simplify ONNX model
            opset=12  # ONNX opset version
        )
        
        print(f"\nModel exported successfully!")
        print(f"  ONNX model saved to: {onnx_path}")
        
        # Get file size
        file_size_mb = os.path.getsize(onnx_path) / (1024 * 1024)
        print(f"  File size: {file_size_mb:.2f} MB")
        
        print("\nONNX model can be used with:")
        print("  - ONNX Runtime (CPU/GPU)")
        print("  - TensorRT (NVIDIA)")
        print("  - OpenVINO (Intel)")
        print("  - Various inference frameworks")
        
    except Exception as e:
        print(f"\nExport failed: {e}")
        print("\nNote: ONNX export requires 'onnx' package:")
        print("  pip install onnx")
else:
    print("No trained model found!")
    print(f"Expected model at: {best_model_path}")
    print("\nPlease train the model first (Section 6) before exporting.")
    print("The model will be saved after training completes.")

## Summary

This notebook demonstrates how to train a custom YOLOv8 model on your own dog dataset for specialized detection tasks.

### Key Points
- Fine-tunes YOLOv8 on custom labeled data
- Requires proper dataset preparation in YOLO format
- Provides training metrics and evaluation
- Supports inference on images and videos with trained model

### Dataset Requirements
1. **Images**: JPG/PNG format in train/val/test directories
2. **Labels**: YOLO format .txt files (one per image)
3. **Structure**: Organized in `data/images/` and `data/labels/` subdirectories
4. **Format**: `class_id center_x center_y width height` (normalized 0-1)

### Training Tips
- Start with a smaller model (yolov8n.pt) for faster training
- Use GPU for significantly faster training times
- Monitor training metrics to avoid overfitting
- Adjust batch_size based on available GPU memory
- Use data augmentation (built into YOLO training)

### Next Steps
- Experiment with different model sizes (n, s, m, l, x)
- Adjust training hyperparameters for better performance
- Collect more diverse training data if accuracy is low
- Export model for deployment (ONNX, TensorRT, etc.)

### Useful Resources
- [Ultralytics YOLO Documentation](https://docs.ultralytics.com/)
- [YOLO Training Guide](https://docs.ultralytics.com/modes/train/)
- [Dataset Preparation Tips](https://docs.ultralytics.com/datasets/)

---

**Project**: YOLO Dog Detection - Custom Training  
**Date**: 2025  
**Model**: YOLOv8 Fine-tuned

## 11. Run Inference with Trained Model

Execute inference using your trained custom model on images from a folder or live camera feed.

In [25]:
# Load the best trained model
best_model_path = MODELS_DIR / 'yolo_dog_custom' / 'weights' / 'best.pt'

if not best_model_path.exists():
    print("No trained model found!")
    print(f"Expected model at: {best_model_path}")
    print("\nPlease train the model first (Section 6) before running inference.")
else:
    print("="*60)
    print("Custom Model Inference")
    print("="*60)
    print(f"\nLoaded model from: {best_model_path}")
    inference_model = YOLO(str(best_model_path))
    print(f"Model ready for inference!")
    
    # Choose inference mode
    print("\n" + "-"*60)
    print("Select Inference Mode:")
    print("-"*60)
    print("\nOption 1: Process images from a folder")
    print("Option 2: Real-time camera detection")
    print("\nTo run inference, uncomment ONE of the options below:")
    print("-"*60)
    
    # ============================================================
    # OPTION 1: Process Images from Folder
    # ============================================================
    # Uncomment the block below to process images from a folder
    
    inference_folder = PROJECT_ROOT / 'test'  # Change to your folder path
    output_folder = OUTPUTS_DIR / 'inference_results'
    output_folder.mkdir(exist_ok=True)
    
    # Get all images in folder
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    images_to_process = []
    for ext in image_extensions:
        images_to_process.extend(inference_folder.glob(ext))
    
    if len(images_to_process) == 0:
        print(f"\nNo images found in {inference_folder}")
        print("Please add images to the folder and try again.")
    else:
        print(f"\nProcessing {len(images_to_process)} images from: {inference_folder}")
        print(f"Saving results to: {output_folder}")
        print("-"*60)
        
        for img_path in tqdm(images_to_process, desc="Processing images"):
            # Run inference
            results = inference_model(
                str(img_path),
                conf=config['confidence_threshold'],
                iou=config['iou_threshold'],
                verbose=False
            )
            
            # Save annotated image
            num_detections = len(results[0].boxes)
            if num_detections > 0:
                output_path = output_folder / f"detected_{img_path.name}"
                annotated = results[0].plot()
                cv2.imwrite(str(output_path), annotated)
                print(f"  {img_path.name}: {num_detections} dog(s) detected")
            else:
                # Save even if no detections
                output_path = output_folder / f"no_detection_{img_path.name}"
                annotated = results[0].plot()
                cv2.imwrite(str(output_path), annotated)
        
        print(f"\nProcessing complete! Results saved to: {output_folder}")
    
    # ============================================================
    # OPTION 2: Real-time Camera Detection
    # ============================================================
    # Uncomment the block below to use camera for real-time detection
    
    # camera_index = 0  # Usually 0 for default camera, try 1, 2, etc. if not working
    
    # print(f"\nStarting camera detection (Camera index: {camera_index})...")
    # print("Press 'q' to quit, 's' to save current frame")
    # print("-"*60)
    
    # cap = cv2.VideoCapture(camera_index)
    
    # if not cap.isOpened():
    #     print(f"\nError: Could not open camera {camera_index}")
    #     print("Try changing camera_index to 1, 2, etc.")
    # else:
    #     # Set camera properties
    #     cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    #     cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
    #     frame_count = 0
    #     fps_time = time.time()
    #     fps = 0
        
    #     camera_output_dir = OUTPUTS_DIR / 'camera_captures'
    #     camera_output_dir.mkdir(exist_ok=True)
        
    #     print("Camera opened successfully! Window should appear...")
        
    #     try:
    #         while True:
    #             ret, frame = cap.read()
                
    #             if not ret:
    #                 print("Failed to grab frame")
    #                 break
                
    #             # Run inference
    #             results = inference_model(
    #                 frame,
    #                 conf=config['confidence_threshold'],
    #                 iou=config['iou_threshold'],
    #                 verbose=False
    #             )
                
    #             # Get annotated frame
    #             annotated_frame = results[0].plot()
                
    #             # Calculate FPS
    #             frame_count += 1
    #             if frame_count % 30 == 0:
    #                 fps = 30 / (time.time() - fps_time)
    #                 fps_time = time.time()
                
    #             # Add FPS and detection count to frame
    #             num_detections = len(results[0].boxes)
    #             cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30), 
    #                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    #             cv2.putText(annotated_frame, f"Dogs: {num_detections}", (10, 70), 
    #                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
    #             # Display frame
    #             cv2.imshow('YOLO Dog Detection - Press Q to quit, S to save', annotated_frame)
                
    #             # Handle key presses
    #             key = cv2.waitKey(1) & 0xFF
                
    #             if key == ord('q'):
    #                 print("\nQuitting camera detection...")
    #                 break
    #             elif key == ord('s'):
    #                 # Save current frame
    #                 timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    #                 save_path = camera_output_dir / f"capture_{timestamp}.jpg"
    #                 cv2.imwrite(str(save_path), annotated_frame)
    #                 print(f"Frame saved: {save_path}")
        
    #     except KeyboardInterrupt:
    #         print("\nInterrupted by user")
        
    #     finally:
    #         cap.release()
    #         cv2.destroyAllWindows()
    #         print(f"\nCamera released. Total frames processed: {frame_count}")
    #         if camera_output_dir.exists():
    #             saved_frames = list(camera_output_dir.glob('*.jpg'))
    #             if saved_frames:
    #                 print(f"Saved frames: {len(saved_frames)} in {camera_output_dir}")
    
    print("\n" + "="*60)
    print("Instructions:")
    print("="*60)
    print("1. Uncomment OPTION 1 block to process images from a folder")
    print("2. Uncomment OPTION 2 block to use camera for real-time detection")
    print("3. Run this cell after uncommenting your chosen option")
    print("="*60)

Custom Model Inference

Loaded model from: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/models/yolo_dog_custom/weights/best.pt
Model ready for inference!

------------------------------------------------------------
Select Inference Mode:
------------------------------------------------------------

Option 1: Process images from a folder
Option 2: Real-time camera detection

To run inference, uncomment ONE of the options below:
------------------------------------------------------------

Processing 2 images from: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/test
Saving results to: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/outputs/inference_results
------------------------------------------------------------


Processing images: 100%|██████████| 2/2 [00:00<00:00, 32.04it/s]


Processing complete! Results saved to: /home/joni/Documentos/univali/inteligencia_artificial/yolo-dog/outputs/inference_results

Instructions:
1. Uncomment OPTION 1 block to process images from a folder
2. Uncomment OPTION 2 block to use camera for real-time detection
3. Run this cell after uncommenting your chosen option
